## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia

**The objective** 
* Extract a Falcon 9 launch records HTML table from Wikipedia
* Parse the table and convert it into a Pandas data frame


URL used for data: https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches

In [1]:
# Uncomment to install if packages arent already
#!pip3 install beautifulsoup4
#!pip3 install requests

In [9]:
# Load necessary libraries
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [10]:
# Here we have some helper functions with their descriptions

def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name   

In [11]:
# We use the following data for the falcon 9 and falcon heavy launches updated on the 9th of june 2021 
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

### Request the Falcon9 Launch Wiki page from its URL

In [12]:
# Use HTTP GET method to requst the Falcon 9 launch HTML page as a HTML response
Falcon9_response = requests.get(static_url).text

In [15]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(Falcon9_response)

In [17]:
# Verify the extraction by printing the titles
Falcon9_response.title()

'<!Doctype Html>\n<Html Class="Client-Nojs" Lang="En" Dir="Ltr">\n<Head>\n<Meta Charset="Utf-8"/>\n<Title>List Of Falcon 9 And Falcon Heavy Launches - Wikipedia</Title>\n<Script>Document.Documentelement.Classname="Client-Js";Rlconf={"Wgbreakframes":False,"Wgseparatortransformtable":["",""],"Wgdigittransformtable":["",""],"Wgdefaultdateformat":"Dmy","Wgmonthnames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"Wgrequestid":"87E3Ecbe-A263-47Db-A9E6-58B53Bf5C0E1","Wgcspnonce":False,"Wgcanonicalnamespace":"","Wgcanonicalspecialpagename":False,"Wgnamespacenumber":0,"Wgpagename":"List_Of_Falcon_9_And_Falcon_Heavy_Launches","Wgtitle":"List Of Falcon 9 And Falcon Heavy Launches","Wgcurrevisionid":1063487988,"Wgrevisionid":1027686922,"Wgarticleid":37574004,"Wgisarticle":True,"Wgisredirect":False,"Wgaction":"View","Wgusername":Null,"Wgusergroups":["*"],"Wgcategories":["Source Attribution","All Articles With Dead External Links"

### Extract all column/variable names from the HTML table header

In [15]:
# Collect all relevant column names from the HTML table header
html_tables = soup.find_all('table')

In [16]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

In [46]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
name = first_launch_table.find_all('th')

# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for i in name:
    if extract_column_from_header(i) != None and len(name)>0:
        column_names.append(extract_column_from_header(i))

In [47]:
print(column_names)

['Flight No.', 'Date and time ( )', '', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', '']


### Create a data frame by parsing the launch HTML tables

In [89]:
# Create an empty dictionary with keys from the extracted column names and converting it into a pandas dataframe 
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Now we fill the dictionary. <br>
Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc. <br>
To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:

In [90]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            print(date)
            
            # Time value
            # Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            print(time)
              
            # Booster version
            # Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            print(bv)
            
            # Launch Site
            # Append the launch site into launch_dict with key `Launch site`
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            print(launch_site)
            
            # Payload
            # Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            print(payload)
            
            # Payload Mass
            # Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            print(payload)
            
            # Orbit
            # Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            print(orbit)
            
            # Customer
            # Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            launch_dict['Customer'].append(customer)
            print(customer)
            
            # Launch outcome
            # Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            print(launch_outcome)
            
            # Booster landing
            # Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            print(booster_landing)
            
            # NOTE: beyond 105 i was getting inconsistent number of values if
            # Remove the if statement to check the inconsistancy
            if extracted_row == 105:
                break

1
4 June 2010
18:45
F9 v1.0B0003.1
CCAFS
Dragon Spacecraft Qualification Unit
Dragon Spacecraft Qualification Unit
LEO
SpaceX
Success

Failure
2
8 December 2010
15:43
F9 v1.0B0004.1
CCAFS
Dragon
Dragon
LEO
NASA
Success
Failure
3
22 May 2012
07:44
F9 v1.0B0005.1
CCAFS
Dragon
Dragon
LEO
NASA
Success
No attempt

4
8 October 2012
00:35
F9 v1.0B0006.1
CCAFS
SpaceX CRS-1
SpaceX CRS-1
LEO
NASA
Success

No attempt
5
1 March 2013
15:10
F9 v1.0B0007.1
CCAFS
SpaceX CRS-2
SpaceX CRS-2
LEO
NASA
Success

No attempt

6
29 September 2013
16:00
F9 v1.1B1003
VAFB
CASSIOPE
CASSIOPE
Polar orbit
MDA
Success
Uncontrolled
7
3 December 2013
22:41
F9 v1.1
CCAFS
SES-8
SES-8
GTO
SES
Success
No attempt
8
6 January 2014
22:06
F9 v1.1
CCAFS
Thaicom 6
Thaicom 6
GTO
Thaicom
Success
No attempt
9
18 April 2014
19:25
F9 v1.1
Cape Canaveral
SpaceX CRS-3
SpaceX CRS-3
LEO
NASA
Success

Controlled
10
14 July 2014
15:15
F9 v1.1
Cape Canaveral
Orbcomm-OG2
Orbcomm-OG2
LEO
Orbcomm
Success
Controlled
11
5 August 2014
08:00
F9 v1

In [91]:
# Checking list lenght
print(extracted_row)
print(len(launch_dict['Flight No.']))
print(len(launch_dict['Launch site']))
print(len(launch_dict['Payload']))
print(len(launch_dict['Payload mass']))
print(len(launch_dict['Orbit']))
print(len(launch_dict['Customer']))
print(len(launch_dict['Launch outcome']))
print(len(launch_dict['Version Booster']))
print(len(launch_dict['Booster landing']))
print(len(launch_dict['Date']))
print(len(launch_dict['Time']))

105
105
105
105
105
105
105
105
105
105
105
105


In [92]:
# Create the data frame
df = pd.DataFrame(launch_dict)

In [93]:
# Save into a csv file
df.to_csv('spacex_web_scraped.csv', index=False)